# MiddleWare: Human In The Loop

## Setup

In [1]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

In [2]:
from typing import TypedDict

# Setup your agent runtime context
class Context(TypedDict):
    first: str
    last: str
    

In [3]:
import re

from langgraph.runtime import Runtime, get_runtime
from langchain_core.tools import tool
    
@tool
def execute_sql(query: str) -> str:
    """Execute a SQLite command and return results."""
    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"


In [19]:
SYSTEM = """You are a careful SQLite analyst.

Rules:
- Think step-by-step.
- When you need data, call the tool `execute_sql` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows unless the user explicitly asks otherwise.
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.
- If the database is offline, ask user to try again later without further comment.
"""

In [20]:
from langchain.agents.middleware import HumanInTheLoopMiddleware 
from langchain.agents import create_agent
from langchain_core.messages import SystemMessage
from langgraph.checkpoint.memory import InMemorySaver

agent = create_agent(
    model="openai:gpt-5",
    tools=[execute_sql],
    prompt=SYSTEM,
    checkpointer=InMemorySaver(),
    context_schema=Context,
    middleware=[
        HumanInTheLoopMiddleware( 
            interrupt_on={
                "execute_sql": {"allow_accept": True, "allow_respond": True},  # No editing allowed
            },
            description_prefix="Tool execution pending approval",
        ),
    ],

)

In [22]:
from langgraph.types import Command
question = "What are the names of all the employees?"

config = {"configurable": {"thread_id": "1"}} 

result = agent.invoke(
    {"messages": [{"role": "user", "content": question}]},
    context= {"first": "Frank", "last": "Harris"},
    config=config
)

if '__interrupt__' in result:
    print(f"\033[1;3;31m{80*'-'}\033[0m")
    print(f"\033[1;3;31m Interrupt:{result['__interrupt__'][-1].value[-1]['description']}\033[0m")

    result = agent.invoke(
        Command( 
            #resume=[{"type": "accept"}]  
            resume=[{"type": "response", "args": "the database is offline."}]  # or "edit", "respond"
     ), 
        config=config # Same thread ID to resume the paused conversation
    )
    print(f"\033[1;3;31m{80*'-'}\033[0m")
           
print(result["messages"][-1].content)

--------------------------------------------------------------------------------
 Interrupt:Tool execution pending approval

Tool: execute_sql
Args: {'query': "SELECT name FROM sqlite_master WHERE type = 'table' ORDER BY name;"}
--------------------------------------------------------------------------------
The database appears to be offline. Please try again later.
